# Numerical Analysis

##### Code Init

In [ ]:
# Function file import
import functionfile_speedygreedy as ff

## Repository Overview

Experiment parameters are created in *experiment_parameters.csv* (works with MS Excel)

### Parameters:
- *experiment_no* : Ensure that every new row has a new number to prevent data overwrites
- *test_model* and *test_parameter*: These define the experiment model and control parameter

| Exp No | test_model                          | Test Description                                                                                                                              | test_parameter Effect                                    |
|:------:|:-----------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------|
|  1.1   | fixed_vs_self_tuning                      | compare fixed architecture to self-tuning architecture for a generated network                                                                | number of changes permitted for self_tuning architecture |
|  1.2   | statistics_fixed_vs_self_tuning           | run fixed_vs_self_tuning for 100 different randomly generated network graphs                                                                   | number of changes permitted for self_tuning architecture |
|  1.3   | statistics_pointdistribution_openloop    | run fixed_vs_self_tuning over initial points on eigenvectors of open-loop dynamics matrix of a single instance of a randomly generated network | number of changes permitted for self_tuning architecture |
|  2.1   | self_tuning_number_of_changes             | compare the effect of 1 vs $n$ number of architecture optimization changes allowed per simulation step                                        | number of changes permitted for self_tuning architecture |
|  2.2   | statistics_self_tuning_number_of_changes  | run self_tuning_number_of_changes for 100 different randomly generated network graphs                                                          | number of changes permitted for self_tuning architecture |
|  3.1   | self_tuning_prediction_horizon            | compare the effect of scaling prediction time horizon                                                                                         | scales the base time horizon                             |
|  3.2   | statistics_self_tuning_prediction_horizon | run self_tuning_prediction_horizon for 100 different randomly generated network graphs                                                         | scales the base time horizon                             | 
|  4.1   | self_tuning_architecture_cost             | compare the effect of scaling architecture running and switching costs                                                                        | scales the base architecture costs                       |
|  4.2   | statistics_self_tuning_architecture_cost  | run self_tuning_architecture_cost for 100 different randomly generated network graphs                                                          | scales the base architecture costs                       |
|  4.3   | self_tuning_architecture_constraints     | compare the effect of tight vs loose architecture constraints on self-tuning architecture under running + switching costs                     | changes lower bound on active architecture sets          |

- Network generation parameters:
    - *number_of_nodes* : number of nodes in the network model
    - *network_model* : type of network connection - generated using *numpy* and *networkx* packages
        - *rand* : random weighted adjacency matrix with weights $[0,1)$
        - *ER* : a realization of an Erdos-Renyi random graph generator with edge probability $p$
        - *BA* : generate a realization of a Barabasi-Albert random graph generator with initial seed $p$
        - *path* : generate a path graph
        - *cycle* : generate a cycle graph
        - *eval_squeeze* : generate spectrum of eigenvalues in $(-1-p, -1+p) \cup (1-p, 1+p)$ to force stable modes with slow convergence and unstable modes with slow divergence
        - *eval_bound* : generate spectrum of eigenvalues in $(-1, -1+p) \cup (1-p, 1)$ to force stable modes with slow convergence
        All graphs are checked to be well-connected, i.e., there are no isolated subnetworks and there is a path (direct or indirect) between every pair of nodes
        For *eval_squeeze* and *eval_bound*, the underlying eigenvectors are randomly generated and normalized. All nodes have a self-connecting edge before uniform scaling is enforced 
    - *network_parameter* : generation parameter for ER, BA, eval_squeeze and eval_bound generators
    - *rho* : uniform scaling parameter to apply to adjacency matrix if required
    - *second_order* and *second_order_network* : specify if network nodes have second-order dynamics and which level is connected by the network adjacency $G_{adj}$
        Dynamics of the $i^{th}$ node is given by $x'_{i,t}$ and $x_{i,t}$
        Open-loop dynamics based on *second_order_network* parameter for states $x_t = \begin{bmatrix} \dots x_{i,t} \dots x'_{i,t} \dots \end{bmatrix}^\top$: 
        - *second_order_network* = 1 : $A = \begin{bmatrix} G_{adj} & \mathbf{0} \\ \mathbf{I} & \mathbf{I} \end{bmatrix}$ where network dynamics affects $x_{i,t}$ and $x'_{i,t+1} = x'_{i,t} + x_{i,t}$
        - *second_order_network* = 2 : $A = \begin{bmatrix} \mathbf{I} & \mathbf{0} \\ \mathbf{I} & G_{adj} \end{bmatrix}$ where network dynamics affects $x'_{i,t}$  and $x'_{i,t+1} = x'_{i,t} + x_{i,t} + \sum_{j\in\mathscr{N}(x_i)}G_{adj,i,j}x_j$
- Architecture parameters: Applicable for both actuators and sensors
    Initializes available architecture as identity basis vectors of $\mathbb{R}^n$ space
    - *initial_architecture_size* : size of initial randomly placed or design-time optimized architecture
    - *architecture_constraint_min*, *architecture_constraint_max* : min and max constraints on the size of the architecture sets. Set equal to each other for constrained optimization
    - *second_order_architecture* : needs to be defined if second-order network dynamics for actuator inputs and sensor measurements
        - *second_order_architecture* = 1 : architecture connected to $x_{i,t}$ nodes
        - *second_order_architecture* = 2 : architecture connected to $x'_{i,t}$ nodes
    - *Q_cost_scaling*, *R_cost_scaling* : scaling costs on states and inputs for control objective - default 1 is identity
    - *B_run_cost*, *C_run_cost* : architecture running costs for actuators and sensors respectively
    - *B_switch_cost*, *C_switch_cost* : architecture switching costs for actuators and sensors respectively
- Disturbance parameters: additive process/measurement noise and un-modelled disturbance generators
    - *W_scaling* : scaling of identity matrix on covariance of zero-mean process noise
    - *V_scaling* : scaling of identity matrix on covariance of zero-mean measurement noise
        Note that *W_scaling* and *V_scaling* are parameters of estimation optimization
    - *disturbance_model* : type of un-modelled noise model
        - *process* - additional noise only in process
        - *measurement* - additional noise only in measurement
        - *combined* - additional noise in both process and measurement
    - *disturbance_step*, *disturbance_number*, *disturbance_magnitude* : how frequently, how many and how large the randomly generated un-modelled noise enters the process and measurements
- Simulation parameters:
    - *prediction_time_horizon* : length of the (MPC-like) receding prediction horizon over which architecture is assumed to be constant and optimized for
    - *X0_scaling* : uniform scaling parameter of the identity covariance matrix of zero-mean distribution over which the initial state is sampled from for all *test_model*(s) other than *statistics_pointdistribution_openloop*. For *statistics_pointdistribution_openloop*, scales the distance of initial state on the open-loop eigenvectors from the unit circle 
    - *multiprocessing* : set to **True** uses *concurrent.futures.ProcessPoolExecutor()* to run statistical experiments for different models/tests in parallel 

# Run + Plot Instructions

Define the experiment parameters in experiment_parameters.csv and set corresponding experiment number to *exp_no*

<span style="color:red;">WARNING: Check for unique experiment numbers to prevent overwriting data.</span>
Default Flags: *run_flag, plot_flag = False, True*
Set *run_flag = True* to regen data and run simulation from scratch
Note that most test cases of a single realization may take 5 - 10 minutes, statistical cases may take 10-20 hours with the *multiprocessing* parameter set to **True** 

#### Code Flags

In [ ]:
run_flag = False    # Uses pre-generated model data from DataDump folder
# run_flag = True   # Simulates experiment and saves data to DataDump folder for plotting

plot_flag = True   # Plots data for exp_no in DataDump folder

# Numerical Analysis and Experiment Summary

## Exp No 1 : Fixed vs Self-tuning Architecture

### Experiment setup

$50$-node undirected, uniformly weighted graph with dynamics $|\lambda_i (A) - 1| \leq 0.1$ $\forall i \in [1, 2,..., 50]$
Architecture constrained by $\mathcal{L}_m = \mathcal{L}_M = \mathcal{L}_{m'} = \mathcal{L}_{M'} = 5$ and no running/switching costs
Design-time fixed active architecture set greedy optimized over a $20$-time step prediction horizon
Self-tuning architecture initialized from design-time greedy optimal active set, optimized at each simulation step over a receding $10$-time step prediction horizon, unrestricted $N\rightarrow\infty$ number of iterations with $N^'=2$ changes to active sets per selection/rejection subsequences

### Code

In [ ]:
exp_no = 1
ff.run_experiment(exp_no, run_check=run_flag, plot_check=plot_flag)

### Discussion

|  Plot  | Fixed Architecture                                                                                                                                  | Self-tuning Architecture                                                                             |
| :---:  |:----------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------|
| Cost Plots | Fails to stabilize the system - predicted and estimated costs increase unbounded                                                                    | Costs converge and system is stabilized within disturbance tolerances                                |
| Trajectory | States increase unbounded and relative magnitude of prediction error marginally stabilises - Suggests sufficient sensors and insufficient actuators | States, state estimates and error converges within disturbance tolerances                            |
| Architecture | Fixed locations and active set size by problem definition                                                                                           | Fixed active set size by problem definition. Greedy optimal change to architecture at each time step |
| Compute time | Only calculates system update parameters at each time step - $ms$/$\mu s$ scale computation                                                        | $\sim 10^1$ seconds per time step to evaluate greedy-optimal architecture                            |

## Exp No 2 : Statistics of Fixed vs Self-tuning Architecture for state-space exploration

### Experiment setup
Goal is exploration of the state space by considering different initial states for a fixed graph
$50$-node undirected, uniformly weighted graph with dynamics $|\lambda_i (A) - 1| \leq 0.1$ $\forall i \in [1, 2,..., 50]$
Architecture constrained by $\mathcal{L}_m = \mathcal{L}_M = \mathcal{L}_{m'} = \mathcal{L}_{M'} = 5$ and no running/switching costs
Design-time fixed active architecture set greedy optimized over a $10$-time step prediction horizon
Self-tuning architecture initialized from design-time greedy optimal active set, optimized at each simulation step over a receding $5$-time step prediction horizon, unrestricted number of iterations with 2 changes to active sets per selection/rejection subsequences
$50$- iterations. For each iteration, the true state is initialized on the scaled unit circle along the $i^{th}$ eigenvector (i.e. normalized eigenvector or unit vector along the $i^{th}$ eigenvector)

### Plot

In [ ]:
exp_no = 2
ff.run_experiment(exp_no, run_check=run_flag, plot_check=plot_flag)

### Discussion
Across all true initial states (and as highlighted by the sample lines), self-tuning architecture consistently performs better than fixed architecture.
On average - there are about $2$ sensor and architecture changes at each time-step

## Exp No 3 : Statistics of Fixed vs Self-tuning Architecture for different graphs

### Experiment setup
$50$-node undirected, uniformly weighted graph with dynamics $|\lambda_i (A) - 1| \leq 0.1$ $\forall i \in [1, 2,..., 50]$
Architecture constrained by $\mathcal{L}_m = \mathcal{L}_M = \mathcal{L}_{m'} = \mathcal{L}_{M'} = 5$ and no running/switching costs
Design-time fixed active architecture set greedy optimized over a $20$-time step prediction horizon
Self-tuning architecture initialized from design-time greedy optimal active set, optimized at each simulation step over a receding $10$-time step prediction horizon, unrestricted $N\rightarrow\infty$ number of iterations with $N^'=2$ changes to active sets per selection/rejection subsequences
$100$- iterations. For each iteration, the network graph is randomized

### Plot

In [ ]:
exp_no = 3
ff.run_experiment(exp_no, run_check=run_flag, plot_check=plot_flag)

### Discussion
For each test case, self-tuning architecture stabilizes the system with significantly lower costs while fixed architecture fails to stabilize the system.

## Exp No 4 : Self-tuning Architecture for number of changes per iteration of the algorithm

### Experiment setup
$50$-node undirected, uniformly weighted graphs with dynamics $|\lambda_i (A) - 1| \leq 0.1$ $\forall i \in [1, 2,..., 50]$
Self-tuning architecture initialized from design-time greedy optimal active set, optimized at each simulation step over a receding $5$-time step prediction horizon, unrestricted number of iterations with 2 changes to active sets per selection/rejection subsequences

### Plot

In [ ]:
exp_no = 4
ff.run_experiment(exp_no, run_check=run_flag, plot_check=plot_flag)

### Discussion
Setting $N\rightarrow\infty$

## Exp No 5 : Statistics of Self-tuning Architecture for number of changes per iteration of the algorithm

### Experiment setup
$50$-node undirected, uniformly weighted graphs with dynamics $|\lambda_i (A) - 1| \leq 0.1$ $\forall i \in [1, 2,..., 50]$
Self-tuning architecture initialized from design-time greedy optimal active set, optimized at each simulation step over a receding $5$-time step prediction horizon, unrestricted number of iterations with 2 changes to active sets per selection/rejection subsequences
$100$- iterations. For each iteration, the number of architecture changes permitted per time-step $N=1$ compared to $N\rightarrow\infty$

### Plot

In [ ]:
exp_no = 5
ff.run_experiment(exp_no, run_check=run_flag, plot_check=plot_flag)

### Discussion
Setting $N\rightarrow\infty$